In [1]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics
import datetime
from finta import TA

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [3]:
periods_per_day = 10
fn = "qm_data_tufv.csv"
df = pd.read_csv(fn, sep=',', na_values=['-1'], index_col=False)

target_col = 'fut'
df['Date'] = pd.to_datetime(df['Date'])
df['CosHour'] = np.cos(2*np.pi*df['Date'].dt.hour/24)
df['SinHOur'] = np.sin(2*np.pi*df['Date'].dt.hour/24)
col = df.pop(target_col)
df.insert(0, col.name, col)
df_f = df.drop(['Sum', 'Date'], axis=1).dropna()
df_train = df_f.iloc[:6000, :]
df_test = df_f.iloc[6000:, :]

In [4]:
def to_sequences(seq_size, obs, target_col):
    x = []
    y = []

    for i in range(len(obs)-seq_size):
        window = obs.iloc[i:(i+seq_size), target_col:].values
        after_window = obs.iloc[i+seq_size, target_col]
        window = [[x] for x in window]
        x.append(window)
        y.append(after_window)
        
    x_train = np.array(x)
    y_train = np.array(y)
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[3]))
    
    return x_train, y_train

x_train = df_train.drop(target_col, axis=1)
y_train = df_train[target_col]

x_test = df_test.drop(target_col, axis=1)
y_test = df_test[target_col]

y_train = y_train.to_numpy().reshape((-1,1))
y_test = y_test.to_numpy().reshape((-1,1))

x_train, y_train = to_sequences(50, df_train, 0)
x_test, y_test = to_sequences(50, df_test, 0)

x_train.shape

(5950, 50, 21)

In [ ]:
input_shape = x_train.shape[1:]
start = datetime.datetime.now()
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
)
finish = datetime.datetime.now()
print("time to execute = ",finish-start)

In [ ]:
model.evaluate(x_test, y_test, verbose=1)
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

In [ ]:
CAPITAL = 3.27
# Get the baseline capital
def backtest(pred, y_test, df, periods_per_day, target_col):
    total   = 0
    pnls    = []
    counter = 0

    df = df.set_index('Date').copy()

    idx = list(df[target_col].index)
    idx_n = []

    for x in range(len(pred)):
        if pred[x][0] > 0.0:
            total += y_test[x]
        elif pred[x][0] < 0.0:
            total -= y_test[x]
        if (counter % periods_per_day == 0):
            pnls.append(total)
            idx_n.append(idx[counter])
        counter += 1

    rets = pd.DataFrame(data=pnls, columns=['cum_pnl'], index=idx_n)
    rets['pct_pnl'] = (rets['cum_pnl'] + CAPITAL).pct_change()
    rets = rets.dropna()

    return rets

rets = backtest(pred, y_test, df, periods_per_day, target_col)

In [ ]:
import matplotlib.pyplot as plt

def perf_summ(
    data: pd.DataFrame, adj: int = 12, title="Metric"
) -> pd.DataFrame:
    summary = pd.DataFrame(
        data=data.mean() * adj, index=[title], columns=["Annualized Return"]
    )
    summary["Annualized Volatility"] = data.std() * np.sqrt(adj)
    summary["Annualized Sharpe Ratio"] = (
        summary["Annualized Return"] / summary["Annualized Volatility"]
    )
    summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
        (data[data < 0]).std() * np.sqrt(adj)
    )

    summary["Skewness"] = data.skew()
    summary["Kurtosis"] = data.kurtosis()
    summary["VaR (0.05)"] = data.quantile(0.05)
    summary["CVaR (0.05)"] = data[data <= data.quantile(0.05)].mean()
    summary["Min"] = data.min()
    summary["Max"] = data.max()

    wealth_index = 1000 * (1 + data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary["Max Drawdown"] = drawdowns.min()
    summary["Calmar Ratio"] = data.mean()/drawdowns.min()

    return summary.T

display(perf_summ(rets['cum_pnl'], adj=252, title="Window=50"))


plt.plot(rets['cum_pnl'])
plt.plot(rets['cum_pnl'].cummax())
plt.xticks(ticks=rets.index[::15], rotation=45)
plt.show()